## Solution 1 — Two-pointer after sorting (recommended)

### Intuition and Approach
Sort the array first. For each index `i`, we look for pairs `(j, k)` with `j > i` and `k > j` such that `nums[i] + nums[j] + nums[k] == 0`. After sorting, we can use the two-pointer technique for the subarray to the right of `i`: start `j = i+1` and `k = n-1` and move pointers inward depending on the sum.

This reduces the 3-sum problem to O(n^2) time by using the sorted order to skip impossible sums and duplicates efficiently.

### Code explanation (detailed)
- `nums.sort()` sorts the array in non-decreasing order.
- Loop `for i in range(n):` chooses the first element of the triplet. The `if i != 0 and nums[i] == nums[i-1]: continue` skipping prevents duplicate triplets that start with the same `nums[i]`.
- Initialize two pointers `j = i+1` and `k = n-1` and while `j < k` compute `total_sum = nums[i] + nums[j] + nums[k]`.
  - If `total_sum < 0`: increase `j` to make sum larger.
  - If `total_sum > 0`: decrease `k` to make sum smaller.
  - If `total_sum == 0`: record triplet `[nums[i], nums[j], nums[k]]` into answer, then move both pointers (`j += 1`, `k -= 1`) and skip duplicates by advancing while equal to previous values.

### Detailed dry run (comprehensive): `nums = [-1,0,1,2,-1,-4]` (example with duplicates and positive/negative mix)
Sorted array -> `nums = [-4, -1, -1, 0, 1, 2]`. Let n = 6. We'll trace each `i` and pointer movements in detail.

i = 0 -> nums[i] = -4
- Set j = 1 (nums[j] = -1), k = 5 (nums[k] = 2). Compute sum = -4 + (-1) + 2 = -3 (< 0) -> need larger sum -> increment j to 2.
- j = 2 (nums[j] = -1), k = 5 -> sum = -4 + (-1) + 2 = -3 -> j -> 3.
- j = 3 (nums[j] = 0), k = 5 -> sum = -4 + 0 + 2 = -2 -> j -> 4.
- j = 4 (nums[j] = 1), k = 5 -> sum = -4 + 1 + 2 = -1 -> j -> 5. Now j == k so inner loop ends. No triplet with i=0.

i = 1 -> nums[i] = -1 (first -1)
- Skip check: i != 0 and nums[1] == nums[0]? No (-1 != -4) so proceed.
- Set j = 2 (nums[j] = -1), k = 5 (nums[k] = 2). sum = -1 + (-1) + 2 = 0 -> found triplet [-1, -1, 2]. Append to ans.
- After finding triplet, increment j -> 3 and decrement k -> 4. Now skip duplicates: while j < k and nums[j] == nums[j-1] -> nums[3]==nums[2]? 0 == -1 false. For k: while j < k and nums[k] == nums[k+1] -> nums[4]==nums[5]? 1 == 2 false. Continue inner loop.
- Now j = 3 (0), k = 4 (1): sum = -1 + 0 + 1 = 0 -> found triplet [-1,0,1]. Append to ans.
- Move j -> 4, k -> 3. Now j >= k so inner loop ends.

i = 2 -> nums[i] = -1 (second -1)
- Check duplicate start: i != 0 and nums[2] == nums[1]? True (-1 == -1), so `continue` to avoid duplicate triplets starting with same value. Skip i=2.

i = 3 -> nums[i] = 0
- j = 4 (1), k = 5 (2): sum = 0 + 1 + 2 = 3 (>0) -> decrement k -> 4. Now j == k stop inner loop.

i = 4 -> nums[i] = 1
- j = 5, k = 5 -> j < k false, skip.

i = 5 -> nums[i] = 2 -> j would be 6 out of range, skip.

Final answer assembled: `[[-1, -1, 2], [-1, 0, 1]]`. The algorithm found both distinct triplets and skipped duplicates correctly.

### Edge cases & notes
- Arrays with fewer than 3 elements -> return empty list.
- All zeros (`[0,0,0,0]`) -> should return `[[0,0,0]]`; duplicate skipping logic yields only one triple.
- Negative and positive values are handled uniformly due to sorting.

### Time and Space complexity (exact)
- Time: sorting costs O(n log n). The main loop runs `i` from 0..n-1 and for each `i` the two-pointer scan is O(n) in total, giving O(n^2) after sorting. Exact overall time: O(n log n + n^2) = O(n^2).
  - Best/Average/Worst: Theta(n^2) dominating term (sorting is lower order).
- Space: O(1) extra (excluding output). Sorting may require O(log n) recursion stack depending on implementation; answer storage is output-dependent.

### When to prefer this solution
- This is the standard efficient approach for 3Sum in interviews and contests. It balances speed and simplicity and handles duplicates elegantly.

In [ ]:
from typing import List
class Solution:
    def threeSum(self, nums: List[int]) -> List[List[int]]:
        ans = []
        n = len(nums)
        nums.sort()
        for i in range(n):
            if i != 0 and nums[i] == nums[i - 1]:
                continue

            # moving the 2 pointers
            j = i + 1
            k = n - 1
            while j < k:
                total_sum = nums[i] + nums[j] + nums[k]
                if total_sum < 0:
                    j += 1
                elif total_sum > 0:
                    k -= 1
                else:
                    temp = [nums[i], nums[j], nums[k]]
                    ans.append(temp)
                    j += 1
                    k -= 1
                    # skip the duplicates if occurred
                    while j < k and nums[j] == nums[j - 1]:
                        j += 1
                    while j < k and nums[k] == nums[k + 1]:
                        k -= 1

        return ans

nums = [-1,0,1,2,-1,-4]
s = Solution()
s.threeSum(nums)

## Solution 2 — Hashset per first index (no sorting required but needs deduplication)

### Intuition and Approach
For each index `i`, we look for pairs `(j, k)` with `j > i` such that `nums[j] + nums[k] == -nums[i]`. We can iterate `j` and use a hashset to store seen numbers; when `third = -(nums[i] + nums[j])` is in the hashset it means we found a valid triplet. To avoid duplicates, we sort the triplet and store it in a set of tuples.

This approach avoids sorting the whole array but uses extra set space to record results and to detect previously seen elements. Worst-case time is O(n^2) and requires O(n^2) space for result deduplication in pathological cases.

### Code explanation (detailed)
- Outer loop `for i in range(n):` picks the first element.
- For each `i`, initialize empty `hashset = set()`.
- For `j` in `i+1..n-1`: compute `third = -(nums[i] + nums[j])`. If `third` is already in hashset, we found a triplet `(nums[i], nums[j], third)`. Sort it and add `tuple(temp)` to `result` set to avoid duplicates. Then always `hashset.add(nums[j])` so future `j` iterations can match earlier values.
- At the end, convert `result` set to a list and return.

### Detailed dry run (comprehensive): `nums = [-1,0,1,2,-1,-4]` (no initial sorting)
We'll trace the outer loop `i` and inner steps, showing how `hashset` evolves and when triplets are detected. The array as given: `[-1,0,1,2,-1,-4]` with indices 0..5.

i = 0 (nums[i] = -1):
- Initialize hashset = {} (empty), result = set().
- j = 1 -> nums[j] = 0 -> third = -(-1 + 0) = 1 -> 1 not in hashset -> add nums[j] (0) to hashset -> hashset = {0}.
- j = 2 -> nums[j] = 1 -> third = -(-1 + 1) = 0 -> 0 is in hashset -> found triplet [-1,1,0] -> sort -> [-1,0,1] -> add tuple(-1,0,1) to result. Then add nums[j] (1) to hashset -> hashset = {0,1}.
- j = 3 -> nums[j] = 2 -> third = -(-1 + 2) = -1 -> -1 not in hashset -> add 2 to hashset -> hashset = {0,1,2}.
- j = 4 -> nums[j] = -1 -> third = -(-1 + -1) = 2 -> 2 is in hashset -> found triplet [-1,-1,2] -> sort -> [-1,-1,2] -> add to result. Add nums[j] (-1) to hashset -> hashset = {0,1,2,-1}.
- j = 5 -> nums[j] = -4 -> third = -(-1 + -4) = 5 -> 5 not in hashset -> add -4 to hashset.

After i=0, result contains {(-1,0,1), (-1,-1,2)}.

i = 1 (nums[i] = 0):
- hashset = {} fresh. j=2 -> nums[j]=1 -> third = -(0+1)=-1 -> -1 not in hashset -> add 1. j=3 -> nums[j]=2 -> third=-2 -> add 2. j=4 -> nums[j]=-1 -> third=1 -> 1 in hashset -> found [0,-1,1] -> sorted [-1,0,1] -> add to result (already present). Continue adding seen values. The `result` set de-duplicates duplicates automatically.

Other i values will be processed similarly; duplicates will not create new entries in `result` because it's a set of tuples.

Final answer (converted to list) -> `[[ -1, 0, 1 ], [ -1, -1, 2 ]]` (order may vary since sets are unordered).

### Edge cases & notes
- Because we don't sort `nums`, duplicate triplets can arise which we remove by sorting each triplet and storing it in a set — this adds overhead.
- The approach is sensitive to memory usage if many unique triplets exist.

### Time and Space complexity (exact)
- Time: Outer loop `i` runs n times, inner loop `j` runs O(n) and hash operations are O(1) average, so O(n^2) time. The sorting of each found triplet is O(3 log 3) constant — negligible.
- Space: O(n) for the per-iteration hashset plus O(T) for result set where T is number of unique triplets (worst-case O(n^2)).

### When to prefer this solution
- Use this when you want a straightforward O(n^2) approach without sorting, and when input sizes are moderate. The sorted-two-pointer method is usually more memory- and time-stable in practice.

In [2]:
from typing import List
class Solution:
    def threeSum(self, nums: List[int]) -> List[List[int]]:
        n = len(nums)
        result = set()
        for i in range(n):
            hashset = set()
            for j in range(i + 1, n):
                third = -(nums[i] + nums[j])
                if third in hashset:
                    temp = [nums[i], nums[j], third]
                    temp.sort()
                    result.add(tuple(temp))
                hashset.add(nums[j])

        ans = list(result)
        return ans

nums = [-1,0,1,2,-1,-4]
s = Solution()
s.threeSum(nums)

[(-1, 0, 1), (-1, -1, 2)]

## Solution 3 — Brute-force triple loops (not practical)

### Intuition and Approach
Check every possible triplet `(i, j, k)` with `i < j < k` and add it to the result if `nums[i] + nums[j] + nums[k] == 0`. Use a set to deduplicate triplets (by sorting each found triplet before adding). This is the simplest and most straightforward approach but has cubic time complexity and is only acceptable for very small inputs.

### Code explanation (detailed)
- Triple nested loops: `for i in range(n): for j in range(i+1, n): for k in range(j+1, n):` examine each triplet exactly once.
- If the sum equals zero, sort the triplet and add as tuple to `my_set` to avoid duplicates. After loops, convert back to list of lists.

### Dry run (edge case): small input `nums = [0,0,0]`
- i=0, j=1, k=2 -> sum=0 so add `(0,0,0)` to set. No other triplets exist. Return `[[0,0,0]]`.

### Edge cases & notes
- Works correctly but extremely slow for n > ~50 due to O(n^3) time.
- Use only for debugging or confirming correctness on tiny test cases.

### Time and Space complexity (exact)
- Time: O(n^3) since we examine every combination of three indices.
- Space: O(T) for set of unique triplets, worst-case O(n^3) distinct triplets (though practically bound by number of combinations).

### Possible improvements
- Use the two-pointer method after sorting to reduce to O(n^2).
- Alternatively, use the hashset-per-index approach if sorting is undesirable.

In [1]:
from typing import List
class Solution:
    def threeSum(self, nums: List[int]) -> List[List[int]]:
        my_set = set()
        n = len(nums)

        # check all possible triplets
        for i in range(n):
            for j in range(i + 1, n):
                for k in range(j + 1, n):
                    if nums[i] + nums[j] + nums[k] == 0:
                        temp = [nums[i], nums[j], nums[k]]
                        temp.sort()
                        my_set.add(tuple(temp))

        ans = [list(item) for item in my_set]
        return ans

nums = [-1,0,1,2,-1,-4]
s = Solution()
s.threeSum(nums)

[[-1, 0, 1], [-1, -1, 2]]